In [77]:
# Carga de las librerías Outliers

# Importar librerías básicas
# Libreria para manipulación y análisis de datos
import pandas as pd
# Libreria para operaciones numéricas
import numpy as np
# Libreria para trabajar con rutas de archivos
#from pathlib import Path
# Libreria para crear visualizaciones
#import matplotlib.pyplot as plt
# Libreria para crear visualizaciones estadísticas
#import seaborn as sns

In [78]:
import sys
from pathlib import Path
ROOT = Path.cwd().parent  # sube desde /notebooks hasta la raíz del proyecto
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

In [79]:
from src.config_loader import ConfigLoader

# Instancia la configuración del proyecto
cfg = ConfigLoader()

# Crea carpetas necesarias
cfg.ensure_dirs()

In [80]:
# =========================================
# Tomamos el archivo de coil2000_clean.csv
# =========================================

print("Ruta del archivo limpio:", cfg.paths.csv_output)
print("¿Existe?:", cfg.paths.csv_output.exists())

df_clean = pd.read_csv(cfg.paths.csv_output)

print("Dimensiones:", df_clean.shape)
df_clean.head()

Ruta del archivo limpio: C:\Users\chile\projects\Mlops_proyect-main\data\clean_data\coil2000_clean.csv
¿Existe?: True
Dimensiones: (626, 86)


,MOSTYPE,MAANTHUI,MGEMOMV,MGEMLEEF,MOSHOOFD,MGODRK,MGODPR,MGODOV,MGODGE,MRELGE,...,APERSONG,AGEZONG,AWAOREG,ABRAND,AZEILPL,APLEZIER,AFIETS,AINBOED,ABYSTAND,CARAVAN
0,37,1,2,2,8,0,4,2,4,3,...,0,0,0,1,0,0,0,0,0,0
1,9,1,3,3,3,1,3,2,4,7,...,0,0,0,1,0,0,0,0,0,0
2,33,1,2,3,8,0,7,0,2,7,...,0,0,0,1,0,0,0,0,0,0
3,22,2,3,3,5,0,5,0,4,7,...,0,0,0,1,0,0,0,0,0,0
4,33,2,3,3,8,0,4,2,3,7,...,0,0,0,1,0,827,0,0,0,0


In [81]:
import importlib
import src.analysis_EDA as analisis_EDA
importlib.reload(analisis_EDA)

<module 'src.analysis_EDA' from 'c:\\Users\\chile\\projects\\Mlops_proyect-main\\src\\analysis_EDA.py'>

In [82]:
# 1) Importar (asegúrate de tener ROOT en sys.path como ya lo hiciste)
from src.analysis_EDA import (
    ProjectConfig, AnalysisOrchestrator, plot_corr_with_target
)

# 2) Config del proyecto (ajusta si cambia columnas categóricas o exclusiones fijas)
cfg_proj = ProjectConfig(
    target="CARAVAN",
    cat_cols=("MOSTYPE","MOSHOOFD","MKOOPKLA"),
    extra_exclusions=("MBERBOER",)
)

In [ ]:
# 3) Ejecutar orquestador sobre tu df_clean (ya cargado)
orch = AnalysisOrchestrator(
    config=cfg_proj,
    outlier_k=1.5,
    policy_quantile=0.15,
    corr_threshold=0.3,
    pca_components=15
)

from src.analysis_EDA import AnalysisOrchestrator
result = orch.run(df_clean, manual_drop=['MOSHOOFD','AWAOREG','PWALAND','PBESAUT'], verbose=True) # drops manuales


# 4) Acceder a resultados clave
print("Mantener outliers en variables:", result["keep_outlier_vars"])
print("Exclusión final:", result["exclude_final"])
print("Pares correlacionados (|r| ≥ 0.3):")
print(result["corr_pairs"].head(10))

# 5) Correlación con target sobre df_mod (previo a PCA)
#corr_filtrada = plot_corr_with_target(result["df_mod"], target="CARAVAN", thr=0.1)

# 6) PCA y splits listos
X_train, X_test, y_train, y_test = result["X_train"], result["X_test"], result["y_train"], result["y_test"]
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


Dimensiones de X (antes de PCA): (626, 26)
Variables usadas en PCA (26): ['MOSTYPE', 'MGODGE', 'MRELOV', 'MFALLEEN', 'MFGEKIND', 'MOPLHOOG', 'MOPLMIDD', 'MOPLLAAG', 'MBERHOOG', 'MBERZELF', 'MBERMIDD', 'MBERARBG', 'MBERARBO', 'MSKD', 'MHHUUR', 'MHKOOP', 'MAUT0', 'MZPART', 'MINKM30', 'MINK4575', 'MINK7512', 'MINKGEM', 'MKOOPKLA', 'PPERSAUT', 'PBRAND', 'APERSAUT']
CARAVAN_in_PCA: False
Componentes PCA: 15
Columnas en X_pca: 15
Varianza explicada acumulada (%): 63.44
Mantener outliers en variables: ['AWAPART', 'MAANTHUI', 'MGEMLEEF', 'MGEMOMV', 'MGODRK', 'MSKA', 'PWAPART']
Exclusión final: ['AWAPART', 'MAANTHUI', 'MGEMLEEF', 'MGEMOMV', 'MGODRK', 'MSKA', 'PWAPART', 'MBERBOER', 'MGODPR', 'MGODOV', 'MRELGE', 'MRELSA', 'MFWEKIND', 'MSKB1', 'MSKB2', 'MSKC', 'MAUT1', 'MAUT2', 'MZFONDS', 'MINK3045', 'MINK123M', 'PWABEDR', 'PWALAND', 'PBESAUT', 'PMOTSCO', 'PVRAAUT', 'PAANHANG', 'PTRACTOR', 'PWERKT', 'PBROM', 'PLEVEN', 'PPERSONG', 'PGEZONG', 'PWAOREG', 'PZEILPL', 'PPLEZIER', 'PFIETS', 'PINBOED', 'P